To construct an example:
q >= bold_N=N+1   and large k

examples tabe    | data        |  F_dual FMDS    | H_dual FMDS    |is the intersection of C and C_ort empty

q, bold_N, N, k = 7, 5, 4, 4   |    False        |    False       |         True

q, bold_N, N, k = 7, 7, 6, 5   |     True        |    False       |         False

q,bold_N, N, k = 8,  8, 7, 6   |     True        |    False       |         False

q,bold_N, N, k =11, 9, 8, 7    |     False       |    False       |         True

q, bold_N, N, k = 11, 10, 9, 7 |     True        |    True        |         True

q, bold_N, N, k = 11, 11, 10, 7|     True        |    False       |         False

q, bold_N, N, k = 11, 11, 10, 8|     True        |    False       |         False

q, bold_N, N, k = 11, 10, 9, 8 |     True        |    True        |         True



q, bold_N, N, k = 13, 9, 8, 7  |     False       |    False       |        True

q, bold_N, N, k = 13, 10, 9, 8 |     False       |    False       |        True

q, bold_N, N, k = 13, 11, 10, 8|     False       |    False       |        True

q, bold_N, N, k = 16, 11, 10, 8|     False       |    False       |        True

q, bold_N, N, k = 16, 12, 11, 9|     False       |    False       |        False




Conjecture:
When q==bold_N the F-dual of FMDS is FMDS







In [178]:
# Helper functions


def F_Weight(v,F): #Usable for all f-metrics in general
    #v must be a vector and F a matrix in the correct field having as rows the spanning family
    #the F_wwight of v is the coset weight of the counter image of v
    
    phi = linear_transformation(F)
    C = F.kernel()
    
    #counterimage of v
    x = phi.lift(v)
    
    #calculate minimum weight
    m = x.hamming_weight()
    
    for c in C:
        y = x + c
        m_y = y.hamming_weight()
        
        if m_y < m:
            m = m_y
    
    return m

def F_MinimumDistance(Gamma,F): #Usable for all f-metrics in general
    #Gamma must be a linear code, F as above 
    #it's better to calculate the various spans of F and see when an element it's in them
    m = F.nrows()
    
    
    for x in Gamma: #how do i only check the projective elements? this can be hard to calculate for large codes!
        
        m_x = F_Weight(x,F)
        print("new x, m_x and m", x, m_x, m) #debug
        if m_x < m and not(x == 0):
            m = m_x
        if m==1:
            return m
        
    return m


def Is_FMDS(Gamma, F):  #For Phase rotation metric, or if u_F is known
    #Gamma must be a linear code, 
   #I assume F is of a Phase metric, u_F distribution otherwise 
    
    if Gamma.dimension() == 0:
        return True
    else:
        bold_N = F.nrows()
        N = F.ncols()
        u=list(range(0,N+1))
        k = Gamma.dimension()
        d = F_MinimumDistance(Gamma, F)
        print(u)
        print("d, N,  u(d-1), k are:",d,N,u[d-1],k)
    
        return k == N - u[d-1]
    


In [179]:
# parameters


q, bold_N, N, k = 7, 5, 4, 4 


Fq.<a> = GF(q, modulus="primitive")


evaluation_points = Fq.list()[:bold_N]; evaluation_points

print(evaluation_points)

# To get an FMDS code from the image of an MDS the following must be true: d_H(D)<=floor(d_H(C)-1/2)
# In such case gamma should always be and FMDS with this program
print(bold_N-k+1 <= floor((bold_N-1)/2),bold_N-k+1,floor((bold_N-1)/2))

[0, 1, 2, 3, 4]
True 2 2


In [180]:
# map phi with parent code as kernel
# check if the intersection is empty

phi_mat = Matrix(Fq, matrix.identity(N).rows() + [-1*ones_matrix(1, N).rows()[0]])
phi = linear_transformation(phi_mat)
C=LinearCode(phi.kernel())
G_C=C.generator_matrix()
G_Cort=C.dual_code().generator_matrix()
print("Is C + C^orth = F_q^(bold N)?",rank(G_C.stack(G_Cort))==bold_N)





Is C + C^orth = F_q^(bold N)? True


In [181]:
# Reed-Solomon (RS) code

D = codes.GeneralizedReedSolomonCode(evaluation_points, k)
G = D.generator_matrix() 
print(G)

[1 1 1 1 1]
[0 1 2 3 4]
[0 1 4 2 2]
[0 1 1 6 1]


In [185]:
# phi(RS-code)

from sage.geometry.hyperplane_arrangement.affine_subspace import AffineSubspace

gamma = LinearCode(G*phi_mat)
G_gamma = gamma.generator_matrix().echelon_form()
print(G_gamma)



print("Is Gamma FMDS?", Is_FMDS(gamma, phi_mat)) #should be yes


[1 0 0 2]
[0 1 0 6]
[0 0 1 5]
new x, m_x and m (0, 0, 0, 0) 0 5
new x, m_x and m (1, 0, 0, 2) 2 5
new x, m_x and m (2, 0, 0, 4) 2 2
new x, m_x and m (3, 0, 0, 6) 2 2
new x, m_x and m (4, 0, 0, 1) 2 2
new x, m_x and m (5, 0, 0, 3) 2 2
new x, m_x and m (6, 0, 0, 5) 2 2
new x, m_x and m (0, 1, 0, 6) 2 2
new x, m_x and m (1, 1, 0, 1) 2 2
new x, m_x and m (2, 1, 0, 3) 3 2
new x, m_x and m (3, 1, 0, 5) 3 2
new x, m_x and m (4, 1, 0, 0) 2 2
new x, m_x and m (5, 1, 0, 2) 3 2
new x, m_x and m (6, 1, 0, 4) 3 2
new x, m_x and m (0, 2, 0, 5) 2 2
new x, m_x and m (1, 2, 0, 0) 2 2
new x, m_x and m (2, 2, 0, 2) 2 2
new x, m_x and m (3, 2, 0, 4) 3 2
new x, m_x and m (4, 2, 0, 6) 3 2
new x, m_x and m (5, 2, 0, 1) 3 2
new x, m_x and m (6, 2, 0, 3) 3 2
new x, m_x and m (0, 3, 0, 4) 2 2
new x, m_x and m (1, 3, 0, 6) 3 2
new x, m_x and m (2, 3, 0, 1) 3 2
new x, m_x and m (3, 3, 0, 3) 2 2
new x, m_x and m (4, 3, 0, 5) 3 2
new x, m_x and m (5, 3, 0, 0) 2 2
new x, m_x and m (6, 3, 0, 2) 3 2
new x, m_x and m (

In [183]:
# RS-code dual

D_dual = D.dual_code()
G_dual = D_dual.generator_matrix().echelon_form()
print(G_dual)




[1 3 6 3 1]


In [184]:
# phi(RS-code dual)

gamma_dual = LinearCode(G_dual*phi_mat)
G_gamma_dual = gamma_dual.generator_matrix().echelon_form()
print(G_gamma_dual)

print("Is the Gamma^d FMDS?", Is_FMDS(gamma_dual, phi_mat))

[0 1 6 1]
new x, m_x and m (0, 0, 0, 0) 0 5
new x, m_x and m (0, 2, 5, 2) 3 5
new x, m_x and m (0, 4, 3, 4) 3 3
new x, m_x and m (0, 6, 1, 6) 3 3
new x, m_x and m (0, 1, 6, 1) 3 3
new x, m_x and m (0, 3, 4, 3) 3 3
new x, m_x and m (0, 5, 2, 5) 3 3
[0, 1, 2, 3, 4, 4]
d, N,  u(d-1), k are: 3 4 2 1
Is the Gamma^d FMDS? False


In [190]:
# gamma_orthogonal

gamma_orthogonal = gamma.dual_code()
G_orthogonal=gamma_orthogonal.generator_matrix().echelon_form()
print(G_orthogonal)
print(gamma_orthogonal.minimum_distance())
print("Is the Orthogonal of Gamma FMDS?", Is_FMDS(gamma_orthogonal, phi_mat))

[1 3 6 3]
4
new x, m_x and m (0, 0, 0, 0) 0 5
new x, m_x and m (1, 3, 6, 3) 3 5
new x, m_x and m (2, 6, 5, 6) 3 3
new x, m_x and m (3, 2, 4, 2) 3 3
new x, m_x and m (4, 5, 3, 5) 3 3
new x, m_x and m (5, 1, 2, 1) 3 3
new x, m_x and m (6, 4, 1, 4) 3 3
[0, 1, 2, 3, 4, 4]
d, N,  u(d-1), k are: 3 4 2 1
Is the Orthogonal of Gamma FMDS? False
